In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [ ]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

In [ ]:
data.describe()

# Предобработка

In [ ]:
answers = {}
# создадим словарь для ответов
data['profit'] = data['revenue'] - data['budget']
# добавляем столбец прибыли - он понадобится для расчетов

# 1. У какого фильма из списка самый большой бюджет?

In [ ]:
answers['1'] = data[data['budget'] == data['budget'].max()]['original_title'].values[0]
# выбираем фильм с максимальным бюджетом и вписываем его название в словарь для ответов 

# 2. Какой из фильмов самый длительный (в минутах)?

In [ ]:
answers['2'] = data[data['runtime'] == data['runtime'].max()]['original_title'].values[0]
# вписываем название самого длительного фильма в словарь для ответов 

# 3. Какой из фильмов самый короткий (в минутах)?





In [ ]:
answers['3'] = data[data['runtime'] == data['runtime'].min()]['original_title'].values[0]

# 4. Какова средняя длительность фильмов?


In [ ]:
answers['4'] = data['runtime'].mean()
# считаем среднюю продолжительность всех фильмов

# 5. Каково медианное значение длительности фильмов? 

In [ ]:
answers['5'] = data['runtime'].median()

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [ ]:
# создаем во время предобработки данных новую колонку 'profit' для упрощения решения
answers['6'] = data[data['profit'] == data['profit'].max()]['original_title'].values[0]
# вписываем название самого прибыльного фильма в словарь для ответов

# 7. Какой фильм самый убыточный? 

In [ ]:
answers['7'] = data[data['profit'] == data['profit'].min()]['original_title'].values[0]

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [ ]:
answers['8'] = data[data['profit'] > 0]['imdb_id'].count()
# считаем идентификаторы всех прибыльных фильмов в словарь для ответов.

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [ ]:
answers['9'] = data.loc[
                        data['release_year'] == 2008
                        ].sort_values(['profit'],ascending=False)['original_title'].values[0]
# выбираем все фильмы за 2008 год, сортируем по прибыльности и первый, самый прибыльный фильм записываем в ответы

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [ ]:
answers['10'] = data.loc[
                        (data['release_year'] >= 2012) &
                        (data['release_year'] <= 2014)
                        ].sort_values(['profit'])['original_title'].values[0]
# все фильмы с 2012 по 2014 года, сортируем по убыточности и первый, самый убыточный фильм записываем в ответы

# 11. Какого жанра фильмов больше всего?

In [ ]:
data_genres = data[['imdb_id', 'genres', 'profit']].copy().set_index('imdb_id')
# создаем датасет с прибылью и жанрами
data_genres.genres = data_genres.genres.str.split('|')
# разделяем жанры, т.к. у многих фильмов несколько жанров
data_gen = data_genres.explode('genres')
# расширяем датасет жанров, для удобства счета
answers['11'] = pd.pivot_table(data_gen, 
                               index='genres', 
                               aggfunc='count').nlargest(1, 'profit').index[0]
# из датасета считаем максимальное количество успешных фильмов для самого производимого жанра
# вносим название жанра в словарь для ответов

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [ ]:
data_pribal = data[data['profit'] > 0]
# выберем прибыльные фильмы
data_genres_pr = data_pribal[['imdb_id', 'genres', 'profit']].copy().set_index('imdb_id')
# создаем датасет с прибылью и жанрами
data_genres_pr.genres = data_genres_pr.genres.str.split('|')
# разделяем жанры, т.к. у многих фильмов несколько жанров
data_gen_2 = data_genres_pr.explode('genres')
# расширяем датасет жанров, для удобства счета
answers['12'] = pd.pivot_table(data_gen_2, 
                               index='genres', 
                               aggfunc='count').nlargest(1, 'profit').index[0]
# из датасета считаем максимальное количество успешных фильмов для самого "надежного" жанра
# вносим название жанра в словарь для ответов

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [ ]:
answers['13'] = data.groupby(['director'])['profit'].sum().sort_values(ascending=False).index[0]

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [ ]:
data_action = data[data.genres.str.contains("Action")] 
# создаем выбираем все фильмы экшн
data_action = data_action['director'].str.cat(sep='|') 
# превращаем всех режисеров в одну строку
data_action = pd.Series(data_action.split('|')) 
# создаем список разделяя строку по '|'
data_action = data_action.value_counts(ascending=False) 
# считаем количество элементов строке
answers['14'] = data_action.index[0] 
# Записываем самого востребованного для этого жанра режиссера, первого в списке

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [ ]:
data_budget = data[['imdb_id', 'cast', 'profit', 'release_year']].copy().set_index('imdb_id')
# создаем датасет с прибылью и актерским составом
data_2012 = data_budget.loc[data_budget['release_year'] == 2012]
# Выбираем фильмы за 2012 год
data_2012.cast = data_2012.cast.str.split('|')
# разделяем актеров, чтобы каждого актера можно было посчитать
data_cast = data_2012.explode('cast')
# расширяем датасет актеров, для удобства счета
answers['15'] = pd.pivot_table(data_cast, 
                               index='cast', 
                               aggfunc='sum').nlargest(1, 'profit').index[0]
# из датасета считаем максимальные сборы, для самого "прибыльного" актера
# вносим имя актера в словарь для ответов

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [ ]:
data_budget_2 = data[['imdb_id', 'cast', 'budget']].copy().set_index('imdb_id')
# создаем датасет с бюджетом и актерским составом
data_budget_2 = data_budget_2[data_budget_2['budget'] > data_budget_2['budget'].mean()]
# создаем датасет с высокобюджетными фильмами
data_budget_2.cast = data_budget_2.cast.str.split('|')
# разделяем актеров, чтобы каждого актера можно было посчитать
data_cast_2 = data_budget_2.explode('cast')
# расширяем датасет актеров, для удобства счета
answers['16'] = pd.pivot_table(data_cast_2, 
                               index='cast', 
                               aggfunc='count').nlargest(1, 'budget').index[0]
# из датасета считаем максимальное количество высокобюджетных фильмов, для самого востребованного актера
# вносим имя актера в словарь для ответов

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [ ]:
data_cast_gen = data[['imdb_id', 'genres', 'cast', 'release_year']].copy().set_index('imdb_id')
# создаем датасет с жанрами и актерским составом
data_cast_gen.genres = data_cast_gen.genres.str.split('|')
# разделяем жанры, на случай если у фильма несколько жанров
data_Cage = data_cast_gen[data_cast_gen['cast'].str.contains("Nicolas Cage")]
# создаем датасет с актером Николасом Кейджем
data_Cage_genres = data_Cage.explode('genres')
# расширяем датасет жанров, чтобы каждый жанр был прописан в датасете
answers['17'] = pd.pivot_table(data_Cage_genres, 
                               index='genres', 
                               aggfunc='count').nlargest(1, 'release_year').index[0]
# из датасета считаем максимальное количество премьер, для самого востребованного зимой режиссера
# вносим имя режиссера в словарь для ответов

# 18. Самый убыточный фильм от Paramount Pictures

In [ ]:
data_paramount = data[data['production_companies'].str.contains("Paramount Pictures")]
# создаем датасет всех фильмом от компании "Paramount Pictures"
answers['18'] = data_paramount[data_paramount['profit'] == data_paramount['profit'].min()]['original_title'].values[0]
# из нового датасета выписываем название самого убыточного фильма в словарь для ответов

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [ ]:
answers['19'] = pd.pivot_table(data, index='release_year', aggfunc='sum').nlargest(1, 'profit').index[0]
# из датасета считаем максимально прибыльный год и вносим результаты словарь для ответов

# 20. Какой самый прибыльный год для студии Warner Bros?

In [ ]:
data_WarnerBros = data[data['production_companies'].str.contains("Warner Bros")]
# создаем датасет c производственной компанией Warner Bros
answers['20'] = pd.pivot_table(data_WarnerBros, 
                               index='release_year', 
                               aggfunc='sum').nlargest(1, 'profit').index[0]
# из датасета считаем максимально прибыльный год для компании Warner Bros вносим результаты словарь для ответов

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [ ]:
data_release = data[['imdb_id', 'original_title', 'release_date']].copy().set_index('imdb_id')
# создаем датасет с названиями фильмов и датой фильма
data_release['release_date'] = data_release['release_date'].str.split('/', expand=True)
# от даты выхода фильма оставляем только месяц премьеры
data_release.rename(columns={"release_date": "release_month"}, inplace=True)
# меняем название колонки с "release_date" (день премьеры) на "release_month" (месяц премьеры)
answers['21'] = pd.pivot_table(data_release, index='release_month', aggfunc='count').nlargest(1, 'original_title').index[0]
# из датасета считаем максимальное количество фильмов, вышедших за месяц
# вносим результаты счетчика в словарь для ответов

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [ ]:
data_release_2 = data[['imdb_id', 'release_date']].copy().set_index('imdb_id')
# создаем датасет с датами выхода фильма
data_release_2.release_date = data_release_2['release_date'].str.split('/', expand=True)
# от даты выхода фильма оставляем только месяц премьеры
data_release_2.rename(columns={"release_date": "release_month"}, inplace=True)
# меняем название колонки с "release_date" (день премьеры) на "release_month" (месяц премьеры)
answers['22'] = data_release_2.loc[data_release_2['release_month'].isin(['6', '7', '8'])].count()[0]
# из датасета считаем количество фильмов, вышедших летом (за 6, 7 и 8 месяц - июнь, июль и август соответствено) 
# вносим результаты счетчика в словарь для ответов

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [ ]:
data_dir_release = data[['imdb_id', 'release_date', 'director']].copy().set_index('imdb_id')
# создаем датасет с датами выхода и режиссерами фильмов
data_dir_release.director = data_dir_release.director.str.split('|')
# разделяем режиссеров, на случай если у фильма несколько режисеров
data_director = data_dir_release.explode('director')
# расширяем датасет режиссеров, чтобы каждый режиссер был с датой премьеры
data_director.release_date = data_director['release_date'].str.split('/', expand=True)
# от даты выхода фильма оставляем только месяц премьеры
data_director.rename(columns={"release_date": "release_month"}, inplace=True)
# меняем название колонки с "release_date" (день премьеры) на "release_month" (месяц премьеры)
data_dir_winter = data_director.loc[data_director['release_month'].isin(['12', '1', '2'])]
# создаем датасет с премьерами зимой (за 12, 1 и 2 месяц - декабрь, январь и февраль соответствено) 
answers['23'] = pd.pivot_table(data_dir_winter, 
                               index='director', 
                               aggfunc='count').nlargest(1, 'release_month').index[0]
# из датасета считаем максимальное количество премьер, для самого востребованного зимой режиссера
# вносим имя режиссера в словарь для ответов

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [ ]:
data_title = data[['imdb_id', 'original_title', 'production_companies']].copy().set_index('imdb_id')
# создаем датасет с названиями фильмов и производственными компаниями
data_title.production_companies = data_title.production_companies.str.split('|')
# разделяем производственные компаниями, на случай если у фильма несколько компаний
data_title_pr = data_title.explode('production_companies')
# расширяем датасет производственных компаний, чтобы каждая компания была с каждым своим фильмом
data_title_pr.original_title = data_title_pr['original_title'].str.len()
# заменяем название фильма длиной названия фильма
data_title_pr.rename(columns={"original_title": "length_title"}, inplace=True)
# меняем название колонки с "original_title" (оригинальное название) на "length_title" (длина названия)
answers['24'] = pd.pivot_table(data_title_pr, 
                               index='production_companies', 
                               aggfunc='mean').nlargest(1, 'length_title').index[0]
# из датасета считаем максимальную среднюю длину названия для производственной компании
# вносим название производственной компании с максимальной средней длиной названия в словарь ответов

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [ ]:
data_overview = data[['imdb_id', 'overview', 'production_companies']].copy().set_index('imdb_id')
# создаем датасет с описанием фильмов и производственными компаниями
data_overview.production_companies = data_overview.production_companies.str.split('|')
# разделяем производственные компаниями, на случай если у фильма несколько компаний
data_overview_pr = data_overview.explode('production_companies')
# расширяем датасет производственных компаний, чтобы каждая компания была с каждым своим описанием к фильму
data_overview_pr.overview = data_overview_pr.overview.str.replace("-", " ")
data_overview_pr.overview = data_overview_pr.overview.str.replace("- ", "")
# заменяем дефис пробелом, а тире с пробелом пустотой, чтобы не считать тире за слово
data_overview_pr.overview = data_overview_pr['overview'].str.count(' ') + 1
# заменяем описание фильма количеством слов в описании к фильма, считая, что слова разделены пробелом
# +1 - это последнее слово, т.к. пробелов в описании на 1 меньше, чем слов
data_overview_pr.rename(columns={"overview": "overview_word_count"}, inplace=True)
# меняем название колонки с "overview" (описание) на "overview_word_count" (количество слов в описании)
answers['25'] = pd.pivot_table(data_overview_pr,
                               index='production_companies',
                               aggfunc='mean').nlargest(1, 'overview_word_count').index[0]
# из датасета считаем максимальную среднее количество слов в описании для производственной компании
# вносим название производственной компании в словарь ответов

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [ ]:
data_vote = data[['imdb_id', 'original_title', 'vote_average']].copy().set_index('imdb_id')
# создаем датасет с названием фильмов и средними оценками
best_1pr = round(data_vote['vote_average'].count()/100)
# создаем переменную и считаем сколько фильмов входит в 1 процент, округляя до целого
answers['26'] = data_vote.groupby(['original_title'])['vote_average'].max().sort_values(ascending=False).head(best_1pr)
# создаем датасет, сортируя по максимальной оценке и выбирая 1 процент самых рейтинговых фильмов вносим датасет в словарь ответов

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [ ]:
data_cast_film = data[['imdb_id', 'original_title', 'cast']].copy().set_index('imdb_id')
# создаем датасет с названием фильмов и актерским составом
data_cast_film.cast = data_cast_film.cast.str.split('|')
# разделяем имена актеров
pairs = Counter()
# создаем счетчик экранных дуэтов
for i in data_cast_film['cast']:    # цикл для актерского состава в каждом фильме
    for j in combinations(i, 2):    # цикл комбинации двух актеров в актерском составе
        if j not in pairs:
            pairs[j] = 1       # создать счетчик для пары, если её нет в счетчике
        else:
            pairs[j] += 1      # увеличить счетчик для пары, если она уже есть в счетчике
answers['27'] = pairs.most_common(1)[0][0]
# выбираем самую частый дуэт актеров в фильме и вносим его в словарь ответов

# Submission

In [ ]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

In [ ]:
# и убедиться что ни чего не пропустил)
len(answers)